# Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import SimpleRNN, GRU
from tensorflow.keras.layers import Dropout, InputLayer, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지
sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

#GPU 사용 설정, -1이면 CPU 사용
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야 함
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# 시계열 모델

In [2]:
# 설정한 모델에 따라 학습 및 예측하는 함수
def predict(model_name: str):
    
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):

        # 1. Data load
        x = np.load(path + 'x_(7727,10,4068).npy')
        y = np.load(path + 'y_(7727,1).npy')

        acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []
        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)

        # 2. Crossvalidation
        for idx, (train_index, test_index) in tqdm(enumerate(sss.split(x, y))):
            print(f'{idx+1}번째')
            X_train, y_train = x[train_index,:,:], y[train_index]
            X_test, y_test = x[test_index,:,:], y[test_index]

            # 3. Select Model (One of LSTM, RNN, GRU)
            if model_name == 'LSTM':
                model = LSTM_model()
            elif model_name == 'RNN':
                model = RNN_model()
            elif model_name == 'GRU':
                model = GRU_model()
            
            early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
            model.fit(X_train, y_train, epochs=300, batch_size=516, validation_split=0.25, callbacks=[early_stop])

            y_pred_test = model.predict(X_test)
            y_pred_test[y_pred_test>0.5]=1
            y_pred_test[y_pred_test<=0.5]=0

            acc = accuracy_score(y_test, y_pred_test)
            precision = precision_score(y_test, y_pred_test)
            recall = recall_score(y_test, y_pred_test)
            f1 = f1_score(y_test, y_pred_test)
            roc_auc = roc_auc_score(y_test, y_pred_test)

            acc_list.append(acc)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            auc_list.append(roc_auc)

        print(f'[{model_name}] 정확도 : {np.mean(acc_list):.3f}, Precision : {np.mean(precision_list):.3f}, Recall : {np.mean(recall_list):.3f}, F1 : {np.mean(f1_list):.3f}, roc_auc : {np.mean(auc_list):.3f}')
        result = [model_name, np.round(np.mean(acc_list), 3), np.round(np.mean(precision_list), 3), np.round(np.mean(recall_list), 3), np.round(np.mean(f1_list), 3), np.round(np.mean(auc_list), 3)]
        return result

In [3]:
x = np.load('/project/LSH/x_(7727,10,4068).npy')

# LSTM
def LSTM_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm
    
# RNN
def RNN_model():
    rnn = Sequential()
    rnn.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    rnn.add(SimpleRNN(units=128, activation='hard_sigmoid', return_sequences=True))
    rnn.add(SimpleRNN(units=64, activation='hard_sigmoid', return_sequences=True))
    rnn.add(Dropout(0.2))
    rnn.add(SimpleRNN(units=64, activation='hard_sigmoid', return_sequences=True))
    rnn.add(SimpleRNN(units=32, activation='hard_sigmoid', return_sequences=False))
    rnn.add(Dropout(0.2))
    rnn.add(Dense(units=1, activation='sigmoid'))
    rnn.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return rnn
    
# GRU
def GRU_model():
    gru = Sequential()
    gru.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    gru.add(GRU(units=128, activation='hard_sigmoid', return_sequences=True))
    gru.add(GRU(units=64, activation='hard_sigmoid', return_sequences=True))
    gru.add(Dropout(0.2))
    gru.add(GRU(units=64, activation='hard_sigmoid', return_sequences=True))
    gru.add(GRU(units=32, activation='hard_sigmoid', return_sequences=False))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1, activation='sigmoid'))
    gru.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return gru

## LSTM / RNN / GRU
- 폐렴 환자의 사망/생존 예측에 대해서 train, test = 8:2 비율의 10겹 교차검증 방법 이용
- Acc, Precision, Recall, F1, `ROC-AUC` 중에서 `ROC-AUC` 정확도 사용


- [비교 기준인 ROC-AUC에 대한 설명](https://bioinformaticsandme.tistory.com/328)
    - 임상에서 AUC-ROC 곡선은 정상인 및 환자 클래스를 구분하는 모델의 성능 평가로 흔하게 사용됨
    - AUC 값이 1에 가까울수록 클래스를 분류하는 성능이 뛰어남을 의미함
    - AUC의 최솟값은 0.5로, 이 경우 모델의 클래스 분리 능력이 전혀 없음을 뜻함




### Issue
- 기존 논문에서는 `restore_best_weights = True`로 코드 작업을 해서 LSTM이 RNN, GRU보다 ROC-AUC가 높았는데 k-fold 검증을 하려면 False로 지정해주어야 하는 것 아닌가? 


- `restore_best_weights = False`로 지정했을 때의 결과
| Model | Acc | Precision | Recall | F1 | ROC-AUC |
|:-----:|:---:|:---------:|:------:|:--:|:-------:|
| LSTM | 0.747 | 0.799 | 0.783 | 0.791 | 0.737 |
| RNN | 0.753 |	0.794 |	0.804 |	0.799 |	0.738 |
| GRU | 0.749 |	0.797 |	0.792 |	0.794 | 0.737 |

In [4]:
# restore_best_weights = True

rnn_result = predict('RNN')
gru_result = predict('GRU')
lstm_result = predict('LSTM')

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 3s 131ms/step - loss: 0.7109 - acc: 0.5068 - val_loss: 0.6718 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 1s 84ms/step - loss: 0.6864 - acc: 0.6000 - val_loss: 0.6806 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6849 - acc: 0.6019 - val_loss: 0.6696 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 1s 82ms/step - loss: 0.6827 - acc: 0.5802 - val_loss: 0.6698 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6755 - acc: 0.5935 - val_loss: 0.6681 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 1s 81ms/step - loss: 0.6751 - acc: 0.6011 - val_loss: 0.6661 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 1s 84ms/step - loss: 0.6723 - acc: 0.6054 - val_loss: 0.6614 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 1s 87ms/step - loss: 0.6606 - acc:

Epoch 2/300
9/9 [==============================] - 1s 85ms/step - loss: 0.7114 - acc: 0.5113 - val_loss: 0.6752 - val_acc: 0.5957
Epoch 3/300
9/9 [==============================] - 1s 84ms/step - loss: 0.6853 - acc: 0.6043 - val_loss: 0.6904 - val_acc: 0.5957
Epoch 4/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6821 - acc: 0.6106 - val_loss: 0.6807 - val_acc: 0.5957
Epoch 5/300
9/9 [==============================] - 1s 84ms/step - loss: 0.6764 - acc: 0.6028 - val_loss: 0.6745 - val_acc: 0.5957
Epoch 6/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6740 - acc: 0.5955 - val_loss: 0.6741 - val_acc: 0.5957
Epoch 7/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6749 - acc: 0.5961 - val_loss: 0.6744 - val_acc: 0.5957
Epoch 8/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6757 - acc: 0.6058 - val_loss: 0.6750 - val_acc: 0.5957
Epoch 9/300
9/9 [==============================] - 1s 82ms/step - loss: 0.6741 - acc: 0.60

Epoch 33/300
9/9 [==============================] - 1s 84ms/step - loss: 0.3158 - acc: 0.9005 - val_loss: 0.5691 - val_acc: 0.7536
Epoch 34/300
9/9 [==============================] - 1s 84ms/step - loss: 0.3011 - acc: 0.9059 - val_loss: 0.5696 - val_acc: 0.7607
Epoch 35/300
9/9 [==============================] - 1s 86ms/step - loss: 0.2902 - acc: 0.9115 - val_loss: 0.5826 - val_acc: 0.7594
Epoch 36/300
9/9 [==============================] - 1s 85ms/step - loss: 0.2793 - acc: 0.9137 - val_loss: 0.5958 - val_acc: 0.7549
Epoch 37/300
9/9 [==============================] - 1s 84ms/step - loss: 0.2751 - acc: 0.9182 - val_loss: 0.6189 - val_acc: 0.7458
Restoring model weights from the end of the best epoch.
Epoch 00037: early stopping
8번째
Epoch 1/300
9/9 [==============================] - 3s 129ms/step - loss: 0.6849 - acc: 0.5618 - val_loss: 0.6699 - val_acc: 0.6138
Epoch 2/300
9/9 [==============================] - 1s 84ms/step - loss: 0.6835 - acc: 0.6024 - val_loss: 0.6660 - val_acc: 0.6

Epoch 12/300
9/9 [==============================] - 1s 81ms/step - loss: 0.4451 - acc: 0.8035 - val_loss: 0.5095 - val_acc: 0.7574
Epoch 13/300
9/9 [==============================] - 1s 83ms/step - loss: 0.4171 - acc: 0.8214 - val_loss: 0.5037 - val_acc: 0.7646
Epoch 14/300
9/9 [==============================] - 1s 82ms/step - loss: 0.3886 - acc: 0.8291 - val_loss: 0.5168 - val_acc: 0.7594
Epoch 15/300
9/9 [==============================] - 1s 82ms/step - loss: 0.3683 - acc: 0.8483 - val_loss: 0.5318 - val_acc: 0.7626
Epoch 16/300
9/9 [==============================] - 1s 85ms/step - loss: 0.3486 - acc: 0.8570 - val_loss: 0.5192 - val_acc: 0.7633
Epoch 17/300
9/9 [==============================] - 1s 85ms/step - loss: 0.3295 - acc: 0.8647 - val_loss: 0.5328 - val_acc: 0.7678
Epoch 18/300
9/9 [==============================] - 1s 84ms/step - loss: 0.3153 - acc: 0.8725 - val_loss: 0.5447 - val_acc: 0.7646
Epoch 19/300
9/9 [==============================] - 1s 86ms/step - loss: 0.2999 - a

...

Epoch 6/300
9/9 [==============================] - 1s 142ms/step - loss: 0.6750 - acc: 0.6026 - val_loss: 0.6767 - val_acc: 0.5899
Epoch 7/300
9/9 [==============================] - 1s 145ms/step - loss: 0.6744 - acc: 0.6108 - val_loss: 0.6788 - val_acc: 0.5899
Epoch 8/300
9/9 [==============================] - 1s 147ms/step - loss: 0.6680 - acc: 0.6166 - val_loss: 0.6733 - val_acc: 0.5899
Epoch 9/300
9/9 [==============================] - 1s 146ms/step - loss: 0.6641 - acc: 0.6157 - val_loss: 0.6689 - val_acc: 0.5899
Epoch 10/300
9/9 [==============================] - 1s 143ms/step - loss: 0.6535 - acc: 0.6207 - val_loss: 0.6525 - val_acc: 0.5899
Epoch 11/300
9/9 [==============================] - 1s 146ms/step - loss: 0.6274 - acc: 0.6360 - val_loss: 0.6140 - val_acc: 0.6675
Epoch 12/300
9/9 [==============================] - 1s 145ms/step - loss: 0.5813 - acc: 0.7020 - val_loss: 0.5750 - val_acc: 0.7128
Epoch 13/300
9/9 [==============================] - 1s 145ms/step - loss: 0.5308

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 5s 226ms/step - loss: 0.6739 - acc: 0.6039 - val_loss: 0.6710 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 2s 170ms/step - loss: 0.6732 - acc: 0.6095 - val_loss: 0.6708 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 2s 174ms/step - loss: 0.6711 - acc: 0.6119 - val_loss: 0.6707 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6703 - acc: 0.6110 - val_loss: 0.6698 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6668 - acc: 0.6108 - val_loss: 0.6676 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6643 - acc: 0.6125 - val_loss: 0.6571 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6426 - acc: 0.6155 - val_loss: 0.6310 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 2s 173ms/step - loss: 0.5985

Epoch 19/300
9/9 [==============================] - 2s 174ms/step - loss: 0.3172 - acc: 0.8848 - val_loss: 0.5396 - val_acc: 0.7684
Epoch 20/300
9/9 [==============================] - 2s 174ms/step - loss: 0.2944 - acc: 0.8975 - val_loss: 0.5702 - val_acc: 0.7710
Epoch 21/300
9/9 [==============================] - 2s 174ms/step - loss: 0.2799 - acc: 0.9029 - val_loss: 0.5764 - val_acc: 0.7684
Epoch 22/300
9/9 [==============================] - 2s 175ms/step - loss: 0.2644 - acc: 0.9141 - val_loss: 0.5898 - val_acc: 0.7490
Epoch 23/300
9/9 [==============================] - 2s 174ms/step - loss: 0.2555 - acc: 0.9200 - val_loss: 0.6212 - val_acc: 0.7620
Epoch 24/300
9/9 [==============================] - 2s 173ms/step - loss: 0.2383 - acc: 0.9245 - val_loss: 0.6280 - val_acc: 0.7523
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
4번째
Epoch 1/300
9/9 [==============================] - 5s 261ms/step - loss: 0.6900 - acc: 0.5586 - val_loss: 0.6615 - val_a

Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping
9번째
Epoch 1/300
9/9 [==============================] - 5s 225ms/step - loss: 0.6782 - acc: 0.5942 - val_loss: 0.6629 - val_acc: 0.6229
Epoch 2/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6811 - acc: 0.5985 - val_loss: 0.6631 - val_acc: 0.6229
Epoch 3/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6763 - acc: 0.5944 - val_loss: 0.6638 - val_acc: 0.6229
Epoch 4/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6736 - acc: 0.6060 - val_loss: 0.6623 - val_acc: 0.6229
Epoch 5/300
9/9 [==============================] - 2s 174ms/step - loss: 0.6730 - acc: 0.6028 - val_loss: 0.6618 - val_acc: 0.6229
Epoch 6/300
9/9 [==============================] - 2s 174ms/step - loss: 0.6702 - acc: 0.6054 - val_loss: 0.6571 - val_acc: 0.6229
Epoch 7/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6623 - acc: 0.6080 - val_loss: 0.6397 - val_acc: 0.

In [5]:
total_result = [lstm_result, rnn_result, gru_result]
result_df = pd.DataFrame(total_result)
result_df.columns = ['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']
result_df

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,LSTM,0.765,0.811,0.802,0.806,0.754
1,RNN,0.760,0.793,0.821,0.807,0.743
2,GRU,0.764,0.798,0.822,0.810,0.748


In [5]:
# restore_best_weights = False

rnn_result = predict('RNN')
gru_result = predict('GRU')
lstm_result = predict('LSTM')

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 3s 129ms/step - loss: 0.6831 - acc: 0.5784 - val_loss: 0.6754 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6851 - acc: 0.6002 - val_loss: 0.6703 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 1s 86ms/step - loss: 0.6764 - acc: 0.5957 - val_loss: 0.6698 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6735 - acc: 0.6028 - val_loss: 0.6696 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 1s 82ms/step - loss: 0.6721 - acc: 0.6047 - val_loss: 0.6668 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6662 - acc: 0.6091 - val_loss: 0.6624 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 1s 81ms/step - loss: 0.6577 - acc: 0.6108 - val_loss: 0.6491 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6326 - acc:

9/9 [==============================] - 1s 85ms/step - loss: 0.4510 - acc: 0.8028 - val_loss: 0.5197 - val_acc: 0.7510
Epoch 14/300
9/9 [==============================] - 1s 84ms/step - loss: 0.4274 - acc: 0.8117 - val_loss: 0.5135 - val_acc: 0.7594
Epoch 15/300
9/9 [==============================] - 1s 82ms/step - loss: 0.3986 - acc: 0.8309 - val_loss: 0.5319 - val_acc: 0.7568
Epoch 16/300
9/9 [==============================] - 1s 84ms/step - loss: 0.3767 - acc: 0.8481 - val_loss: 0.5278 - val_acc: 0.7587
Epoch 17/300
9/9 [==============================] - 1s 85ms/step - loss: 0.3561 - acc: 0.8574 - val_loss: 0.5385 - val_acc: 0.7587
Epoch 18/300
9/9 [==============================] - 1s 84ms/step - loss: 0.3346 - acc: 0.8667 - val_loss: 0.5518 - val_acc: 0.7542
Epoch 19/300
9/9 [==============================] - 1s 83ms/step - loss: 0.3206 - acc: 0.8794 - val_loss: 0.5726 - val_acc: 0.7613
Epoch 20/300
9/9 [==============================] - 1s 84ms/step - loss: 0.3004 - acc: 0.8878 - 

9/9 [==============================] - 1s 83ms/step - loss: 0.6789 - acc: 0.5892 - val_loss: 0.6622 - val_acc: 0.6229
Epoch 5/300
9/9 [==============================] - 1s 82ms/step - loss: 0.6774 - acc: 0.5892 - val_loss: 0.6593 - val_acc: 0.6229
Epoch 6/300
9/9 [==============================] - 1s 84ms/step - loss: 0.6737 - acc: 0.6032 - val_loss: 0.6579 - val_acc: 0.6229
Epoch 7/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6731 - acc: 0.5920 - val_loss: 0.6544 - val_acc: 0.6229
Epoch 8/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6603 - acc: 0.6104 - val_loss: 0.6445 - val_acc: 0.6229
Epoch 9/300
9/9 [==============================] - 1s 83ms/step - loss: 0.6512 - acc: 0.6132 - val_loss: 0.6224 - val_acc: 0.6229
Epoch 10/300
9/9 [==============================] - 1s 85ms/step - loss: 0.6209 - acc: 0.6550 - val_loss: 0.5871 - val_acc: 0.7232
Epoch 11/300
9/9 [==============================] - 1s 83ms/step - loss: 0.5810 - acc: 0.7023 - val_l

0it [00:00, ?it/s]

...

Epoch 22/300
9/9 [==============================] - 1s 146ms/step - loss: 0.2876 - acc: 0.8906 - val_loss: 0.5711 - val_acc: 0.7626
Epoch 23/300
9/9 [==============================] - 1s 146ms/step - loss: 0.2713 - acc: 0.9027 - val_loss: 0.5962 - val_acc: 0.7555
Epoch 24/300
9/9 [==============================] - 1s 144ms/step - loss: 0.2581 - acc: 0.9068 - val_loss: 0.6133 - val_acc: 0.7471
Epoch 00024: early stopping
10번째
Epoch 1/300
9/9 [==============================] - 5s 196ms/step - loss: 0.6808 - acc: 0.6026 - val_loss: 0.6771 - val_acc: 0.5899
Epoch 2/300
9/9 [==============================] - 1s 141ms/step - loss: 0.6719 - acc: 0.6114 - val_loss: 0.6792 - val_acc: 0.5899
Epoch 3/300
9/9 [==============================] - 1s 145ms/step - loss: 0.6770 - acc: 0.6080 - val_loss: 0.6782 - val_acc: 0.5899
Epoch 4/300
9/9 [==============================] - 1s 143ms/step - loss: 0.6748 - acc: 0.6136 - val_loss: 0.6786 - val_acc: 0.5899
Epoch 5/300
9/9 [==============================

0it [00:00, ?it/s]

1번째
Epoch 1/300
9/9 [==============================] - 5s 224ms/step - loss: 0.6779 - acc: 0.5968 - val_loss: 0.6745 - val_acc: 0.6054
Epoch 2/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6748 - acc: 0.6071 - val_loss: 0.6708 - val_acc: 0.6054
Epoch 3/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6693 - acc: 0.6106 - val_loss: 0.6705 - val_acc: 0.6054
Epoch 4/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6743 - acc: 0.6069 - val_loss: 0.6703 - val_acc: 0.6054
Epoch 5/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6702 - acc: 0.6101 - val_loss: 0.6693 - val_acc: 0.6054
Epoch 6/300
9/9 [==============================] - 2s 174ms/step - loss: 0.6679 - acc: 0.6095 - val_loss: 0.6659 - val_acc: 0.6054
Epoch 7/300
9/9 [==============================] - 2s 174ms/step - loss: 0.6573 - acc: 0.6125 - val_loss: 0.6510 - val_acc: 0.6054
Epoch 8/300
9/9 [==============================] - 2s 175ms/step - loss: 0.6312

Epoch 20/300
9/9 [==============================] - 2s 174ms/step - loss: 0.2765 - acc: 0.9033 - val_loss: 0.5871 - val_acc: 0.7503
Epoch 21/300
9/9 [==============================] - 2s 173ms/step - loss: 0.2627 - acc: 0.9128 - val_loss: 0.6152 - val_acc: 0.7639
Epoch 22/300
9/9 [==============================] - 2s 175ms/step - loss: 0.2554 - acc: 0.9167 - val_loss: 0.6315 - val_acc: 0.7607
Epoch 23/300
9/9 [==============================] - 2s 175ms/step - loss: 0.2440 - acc: 0.9208 - val_loss: 0.6378 - val_acc: 0.7464
Epoch 00023: early stopping
4번째
Epoch 1/300
9/9 [==============================] - 5s 263ms/step - loss: 0.6800 - acc: 0.6009 - val_loss: 0.6665 - val_acc: 0.6274
Epoch 2/300
9/9 [==============================] - 2s 173ms/step - loss: 0.6740 - acc: 0.5961 - val_loss: 0.6610 - val_acc: 0.6274
Epoch 3/300
9/9 [==============================] - 2s 174ms/step - loss: 0.6758 - acc: 0.6019 - val_loss: 0.6606 - val_acc: 0.6274
Epoch 4/300
9/9 [==============================

9/9 [==============================] - 5s 224ms/step - loss: 0.6766 - acc: 0.6050 - val_loss: 0.6631 - val_acc: 0.6229
Epoch 2/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6784 - acc: 0.5972 - val_loss: 0.6646 - val_acc: 0.6229
Epoch 3/300
9/9 [==============================] - 2s 171ms/step - loss: 0.6721 - acc: 0.6035 - val_loss: 0.6624 - val_acc: 0.6229
Epoch 4/300
9/9 [==============================] - 2s 171ms/step - loss: 0.6745 - acc: 0.6037 - val_loss: 0.6638 - val_acc: 0.6229
Epoch 5/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6696 - acc: 0.6039 - val_loss: 0.6621 - val_acc: 0.6229
Epoch 6/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6708 - acc: 0.6052 - val_loss: 0.6583 - val_acc: 0.6229
Epoch 7/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6637 - acc: 0.6067 - val_loss: 0.6451 - val_acc: 0.6229
Epoch 8/300
9/9 [==============================] - 2s 172ms/step - loss: 0.6399 - acc: 0.6129 -

In [6]:
total_result = [lstm_result, rnn_result, gru_result]
result_df = pd.DataFrame(total_result)
result_df.columns = ['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']
result_df

,Model,Acc,Precision,Recall,F1,ROC-AUC
0,LSTM,0.747,0.799,0.783,0.791,0.737
1,RNN,0.753,0.794,0.804,0.799,0.738
2,GRU,0.749,0.797,0.792,0.794,0.737


# 비시계열 모델
- [Support vector machine](https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-2%EC%84%9C%ED%8F%AC%ED%8A%B8-%EB%B2%A1%ED%84%B0-%EB%A8%B8%EC%8B%A0-SVM)

In [3]:
def predict2(model):
    
    path = '/project/LSH/'
    with tf.device('/device:GPU:0'):
        
        # 1. Data load
        x = np.load(path + 'x_(7727,10,4068).npy').reshape(7727,-1)
        y = np.load(path + 'y_(7727,1).npy')

        sss = StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 42)
        acc_list, precision_list, recall_list, f1_list, auc_list = [], [], [], [], []

        # 2. Crossvalidation
        for (train_index, test_index) in tqdm(sss.split(x, y)):
                    
            X_train, y_train = x[train_index,:], y[train_index]
            X_test, y_test = x[test_index,:], y[test_index]
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)
            f1 = f1_score(y_test, pred)
            roc_auc = roc_auc_score(y_test, pred)
            acc = accuracy_score(y_test, pred)
            
            acc_list.append(acc)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            auc_list.append(roc_auc)

        print(f'{str(model)[:-2]} Acc : {np.mean(acc_list):.3f}, Precision : {np.mean(precision_list):.3f}, Recall : {np.mean(recall_list):.3f}, F1 : {np.mean(f1_list):.3f}, roc_auc : {np.mean(auc_list):.3f}')
        return [str(model)[:-2], np.round(np.mean(acc_list), 3), np.round(np.mean(precision_list), 3), np.round(np.mean(recall_list), 3), np.round(np.mean(f1_list), 3), np.round(np.mean(auc_list), 3)]

In [ ]:
# 1. Support vector machine (1-fold당 25분 정도 소요)
# from sklearn.svm import SVC
# model = SVC()
# SVC = predict2(model)

# 2. Gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
GNB = predict2(model)

# 3. Multinomial naive bayes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
MNB = predict2(model)

# 4. Bernoulli naive bayes
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()
BNB = predict2(model)

# 5. K nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
KN = predict2(model)

# 6. Decision Tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
DT = predict2(model)

In [ ]:
# 7. RandomForest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
RF = predict2(model)

In [ ]:
# 8. Gradient boosting (1-fold당 7분 정도 소요)
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
GB = predict2(model)

0it [00:00, ?it/s]

In [ ]:
# 9. Multi-Layer Perceptron
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
MLP = predict2(model)

# 10. AdaBoost
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
AB = predict2(model)

In [6]:
GNB = ['GaussianNB', 0.580, 0.725, 0.502, 0.593, 0.602]
MNB = ['MultinomialNB', 0.692, 0.749, 0.746, 0.747, 0.677]
BNB = ['BernoulliNB', 0.695, 0.785, 0.689, 0.734, 0.696]
KN = ['KNeighbors', 0.651, 0.677, 0.820, 0.741, 0.603]
DT = ['DecisionTree', 0.673, 0.735, 0.726, 0.730, 0.657]
RF = ['RandomForest', 0.755, 0.765, 0.864, 0.811, 0.724]
GB = ['GradientBoosting', 0.753, 0.782, 0.827, 0.804, 0.733]
MLP = ['MLP', 0.749, 0.787, 0.807, 0.797, 0.732]
AB = ['AdaBoost', 0.734, 0.781, 0.785, 0.783, 0.720]

total_result = [GNB, MNB, BNB, KN, DT, RF, GB, MLP, AB]
result_df = pd.DataFrame(total_result)
result_df.columns = ['Model', 'Acc', 'Precision', 'Recall', 'F1', 'ROC-AUC']
result_df = result_df.sort_values(by = 'ROC-AUC', ascending = False)
result_df

,Model,Acc,Precision,Recall,F1,ROC-AUC
6,GradientBoosting,0.753,0.782,0.827,0.804,0.733
7,MLP,0.749,0.787,0.807,0.797,0.732
5,RandomForest,0.755,0.765,0.864,0.811,0.724
8,AdaBoost,0.734,0.781,0.785,0.783,0.720
2,BernoulliNB,0.695,0.785,0.689,0.734,0.696
1,MultinomialNB,0.692,0.749,0.746,0.747,0.677
4,DecisionTree,0.673,0.735,0.726,0.730,0.657
3,KNeighbors,0.651,0.677,0.820,0.741,0.603
0,GaussianNB,0.580,0.725,0.502,0.593,0.602
